In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("database.db")  # This creates a new database file if not already present
cursor = conn.cursor()

# Load CSV into a Pandas DataFrame
orders_df = pd.read_csv("orders1.csv")
delivery_df = pd.read_csv("delivery_performance.csv")

# Save DataFrame to SQLite (ensure column names match your SQL query)
orders_df.to_sql("orders1", conn, if_exists="replace", index=False)
delivery_df.to_sql("delivery_performance", conn, if_exists="replace", index=False)

print("Orders data loaded successfully!")

Orders data loaded successfully!


# Task 1.

In [3]:
query = """
WITH customer_order_count AS (
    SELECT customer_id, COUNT(order_id) AS total_orders, MAX(order_date) AS last_order_date
    FROM orders1
    GROUP BY customer_id
)
SELECT customer_id
FROM customer_order_count
WHERE total_orders >= 2
AND last_order_date <= date('now', '-60 day');
"""

# Execute query and fetch results into a DataFrame
df = pd.read_sql(query, conn)

# Display results
print(df)

# Close the connection
# conn.close()


     customer_id
0           1001
1           1002
2           1003
3           1005
4           1006
..           ...
959         1996
960         1997
961         1998
962         1999
963         2000

[964 rows x 1 columns]


In [4]:
# df.to_csv("customer_orders_results.csv", index=False)
# print("Results saved as customer_orders_results.csv")

# Task 2.

In [5]:
query = """
WITH ordered_data AS (
    SELECT customer_id, order_date,
           LAG(order_date) OVER (PARTITION BY customer_id ORDER BY order_date) AS prev_order_date
    FROM orders1
)
SELECT customer_id, 
       AVG((julianday(order_date) - julianday(prev_order_date))) AS avg_days_between_orders
FROM ordered_data
WHERE prev_order_date IS NOT NULL
GROUP BY customer_id;
"""

In [6]:
df = pd.read_sql(query, conn)

# Save results as CSV
df.to_csv("avg_order_gap_results.csv", index=False)

# Display first few results
print(df.head())

# Close connection
# conn.close()

   customer_id  avg_days_between_orders
0         1001                45.666667
1         1002               225.000000
2         1003               141.500000
3         1005                24.500000
4         1006                95.666667


In [7]:
query = "PRAGMA table_info(orders1);"
df = pd.read_sql(query, conn)
print(df)


   cid          name     type  notnull dflt_value  pk
0    0      order_id  INTEGER        0       None   0
1    1   customer_id  INTEGER        0       None   0
2    2    order_date     TEXT        0       None   0
3    3          city     TEXT        0       None   0
4    4  total_amount     REAL        0       None   0


In [8]:
query = """
ALTER TABLE orders1 ADD COLUMN total_amount_fixed REAL;
UPDATE orders1 SET total_amount_fixed = CAST(total_amount AS REAL);
"""
conn.executescript(query)
conn.commit()


In [9]:
query = "SELECT total_amount FROM orders1 WHERE total_amount NOT LIKE '%0%' AND total_amount NOT LIKE '%1%' AND total_amount NOT LIKE '%2%' AND total_amount NOT LIKE '%3%' AND total_amount NOT LIKE '%4%' AND total_amount NOT LIKE '%5%' AND total_amount NOT LIKE '%6%' AND total_amount NOT LIKE '%7%' AND total_amount NOT LIKE '%8%' AND total_amount NOT LIKE '%9%' LIMIT 10;"
df_invalid = pd.read_sql(query, conn)
print(df_invalid)


Empty DataFrame
Columns: [total_amount]
Index: []


In [10]:
query = """
UPDATE orders1 
SET total_amount = NULL 
WHERE total_amount NOT LIKE '%0%' 
AND total_amount NOT LIKE '%1%' 
AND total_amount NOT LIKE '%2%' 
AND total_amount NOT LIKE '%3%' 
AND total_amount NOT LIKE '%4%' 
AND total_amount NOT LIKE '%5%' 
AND total_amount NOT LIKE '%6%' 
AND total_amount NOT LIKE '%7%' 
AND total_amount NOT LIKE '%8%' 
AND total_amount NOT LIKE '%9%';
"""
conn.executescript(query)
conn.commit()


# Task 3.

In [11]:
query = """
WITH customer_spend AS (
    SELECT customer_id, 
           SUM(CAST(total_amount AS REAL)) AS total_spent, 
           COUNT(order_id) AS total_orders
    FROM orders1
    GROUP BY customer_id
),
percentile_cutoff AS (
    SELECT total_spent 
    FROM customer_spend
    ORDER BY total_spent DESC
    LIMIT 1 OFFSET CAST((SELECT COUNT(*) * 0.1 FROM customer_spend) AS INTEGER)
)
SELECT c.customer_id, 
       c.total_spent, 
       (c.total_spent * 1.0 / c.total_orders) AS avg_order_value
FROM customer_spend c
JOIN percentile_cutoff p 
ON c.total_spent >= p.total_spent
ORDER BY c.total_spent DESC;

"""
df = pd.read_sql(query, conn)
df.to_csv("top_10_percent_customers.csv", index=False)
df.head()


,customer_id,total_spent,avg_order_value
0,1161,3814.541693,293.426284
1,1770,3528.258650,320.750786
2,1578,3473.933656,315.812151
3,1200,3331.480574,302.861870
4,1916,3161.905979,351.322887


# Task 4.

In [13]:
# Checking the schema of the tables to confirm the column types
schema_orders1 = "PRAGMA table_info(orders1);"
schema_delivery_performance = "PRAGMA table_info(delivery_performance);"

# Running the queries to check the column info
orders1_schema = pd.read_sql(schema_orders1, conn)
delivery_performance_schema = pd.read_sql(schema_delivery_performance, conn)

# Displaying the results
print("Orders1 Table Schema:")
print(orders1_schema)

print("\nDelivery Performance Table Schema:")
print(delivery_performance_schema)


Orders1 Table Schema:
   cid                name     type  notnull dflt_value  pk
0    0            order_id  INTEGER        0       None   0
1    1         customer_id  INTEGER        0       None   0
2    2          order_date     TEXT        0       None   0
3    3                city     TEXT        0       None   0
4    4        total_amount     REAL        0       None   0
5    5  total_amount_fixed     REAL        0       None   0

Delivery Performance Table Schema:
   cid             name     type  notnull dflt_value  pk
0    0         order_id  INTEGER        0       None   0
1    1      customer_id  INTEGER        0       None   0
2    2  delivery_status     TEXT        0       None   0
3    3    delivery_time  INTEGER        0       None   0


In [14]:
# Checking if there are any missing or NULL order_id in orders1
missing_order_id_orders1 = "SELECT * FROM orders1 WHERE order_id IS NULL;"
missing_order_id_orders1_result = pd.read_sql(missing_order_id_orders1, conn)

# Checking if there are any missing or NULL order_id in delivery_performance
missing_order_id_delivery_performance = "SELECT * FROM delivery_performance WHERE order_id IS NULL;"
missing_order_id_delivery_performance_result = pd.read_sql(missing_order_id_delivery_performance, conn)

# Displaying the results
print("Missing order_id in orders1:")
print(missing_order_id_orders1_result)

print("\nMissing order_id in delivery_performance:")
print(missing_order_id_delivery_performance_result)


Missing order_id in orders1:
Empty DataFrame
Columns: [order_id, customer_id, order_date, city, total_amount, total_amount_fixed]
Index: []

Missing order_id in delivery_performance:
Empty DataFrame
Columns: [order_id, customer_id, delivery_status, delivery_time]
Index: []


In [15]:
# Sampling data from orders1 table
sample_orders1 = "SELECT order_id, customer_id FROM orders1 LIMIT 10;"
sample_orders1_result = pd.read_sql(sample_orders1, conn)

# Sampling data from delivery_performance table
sample_delivery_performance = "SELECT order_id, delivery_status FROM delivery_performance LIMIT 10;"
sample_delivery_performance_result = pd.read_sql(sample_delivery_performance, conn)

# Displaying the results
print("Sample from orders1 table:")
print(sample_orders1_result)

print("\nSample from delivery_performance table:")
print(sample_delivery_performance_result)


Sample from orders1 table:
   order_id  customer_id
0         1         1102
1         2         1045
2         3         1301
3         4         1252
4         5         1443
5         6         1335
6         7         1262
7         8         1802
8         9         1730
9        10         1194

Sample from delivery_performance table:
   order_id delivery_status
0         1         On Time
1         2         Delayed
2         3         On Time
3         4         On Time
4         5         On Time
5         6         On Time
6         7         On Time
7         8         On Time
8         9         On Time
9        10         On Time


In [16]:
query = """
WITH joined_data AS (
    SELECT o.city, 
           o.order_id, 
           d.delivery_status, 
           d.delivery_time
    FROM orders1 o
    JOIN delivery_performance d 
        ON o.order_id = d.order_id
),
on_time_deliveries AS (
    SELECT city,
           COUNT(order_id) AS total_deliveries,
           SUM(CASE WHEN delivery_status = 'On Time' THEN 1 ELSE 0 END) AS on_time_deliveries
    FROM joined_data
    GROUP BY city
)
SELECT city, 
       total_deliveries, 
       on_time_deliveries, 
       ROUND((on_time_deliveries * 100.0 / total_deliveries), 2) AS on_time_percentage
FROM on_time_deliveries
ORDER BY on_time_percentage DESC;
"""

# Execute the query and load the result into a DataFrame
df = pd.read_sql(query, conn)

# Save the result to a CSV file
df.to_csv("on_time_delivery_report.csv", index=False)

# Display the result
df.head()


,city,total_deliveries,on_time_deliveries,on_time_percentage
0,Houston,974,827,84.91
1,New York,992,839,84.58
2,San Francisco,1048,886,84.54
3,Los Angeles,1013,854,84.30
4,Chicago,973,814,83.66
